# Notes on creating a generic particles reader/writer class

there's soemthing funky going on with references 

In [1]:
DATADIR = '/gpfs/slac/cryo/fs1/g/ML/cryoEM-notebooks/data/20181005-rib-TEM4/Extract/extract_one_2dclass/Micrographs'

In [2]:
import h5py
import glob
from os import path
import numpy as np
import mrcfile

In [3]:
def parse_particle_star( filepath ):
    with open(filepath, 'r') as f:
        keys = []
        for i in f.readlines():
            if i.startswith('_rln'):
                k = i.split()[0].replace('_rln','')
                keys.append(k)
            else:
                data = i.split()
                if len(data) == len(keys) and len(data) > 0:
                    out = {}
                    for n,k in enumerate(keys):
                        out[k] = data[n]
                        try:
                            if k not in ('ClassNumber',):
                                out[k] = float(data[n])
                        except:
                            pass
#                     print("  %s found: %s" % (i,out,))
                    yield out

def get_particle_image( mrcs ):
    with mrcfile.open(mrcs, permissive=True) as particles:
        # print( particles.data.shape )
        # mrcfile returns just a 2 dimensional matrix if only single image
        if len(particles.data.shape) == 2:
            yield particles.data
        else:
            for p in particles.data:
                yield p

def get_images( file_pattern, star_pattern ):
    for mrc in glob.glob( file_pattern ):
#         print("File: %s" % (mrc,))
        star_file = mrc.replace( *star_pattern )
        md = None
        if path.exists( star_file ):
            md = [ m for m in parse_particle_star( star_file ) ]
#         print(" meta: %s -> %s" % (star_file,len(md)))
        for i, p in enumerate( get_particle_image( mrc ) ):
            yield p, md[i] if md else None
                
def create_particle_stack( output_file, file_pattern='./*.mrcs', star_pattern=('.mrcs', '_extract.star'), dsname='particles', parse_particle_star=True, first=None ):
    with h5py.File(output_file, 'w') as hf:
        i = 0
        for p,meta in get_images( file_pattern, star_pattern ):
#             print( " got %s: %s (%s)" % (i,p.shape,meta) )
            if i == 0:
                hf.create_dataset(dsname, data=p[np.newaxis,...], maxshape=(None,p.shape[0],p.shape[1]), chunks=True)
            else:
#                 print("P: %s" % (p,))
                hf[dsname].resize( (hf[dsname].shape[0]+1), axis=0 )
                hf[dsname][-hf[dsname].shape[0]:] = p.copy()

#             print("%s" % (hf[dsname].shape,))
            print( ".", end='')
            
            if first and i >= first - 1:
                break
            i = i+1


In [4]:
create_particle_stack( '/tmp/particles.hdf', file_pattern=DATADIR+"/*.mrcs", first=5 )

.....

hmmm... all the images are the same... even though i'm using p.copy() above...

In [5]:
from matplotlib import pyplot as plt
with h5py.File( '/tmp/particles.hdf', 'r' ) as f:
    print( f['particles'] )
    fig = plt.figure(figsize=(12, 12))
    for i in range( f['particles'].shape[0] ):
        print( "I %s: %s " % (i,f['particles'][i]))
        fig.add_subplot( 6, 6, i+1 )
        plt.imshow( f['particles'][i], cmap='Greys' )
    plt.show()

<HDF5 dataset "particles": shape (5, 540, 540), type "<f4">
I 0: [[-1.6703042   0.04279047  1.1082757  ...  1.1201062  -1.0580032
  -0.18329015]
 [-1.3252518  -0.5656686   0.50778794 ...  1.8512449  -0.5306258
  -0.36595204]
 [-1.1735461  -0.89015764  0.26884112 ...  1.4333687  -0.44683903
  -0.42583343]
 ...
 [-0.9653343   0.10381357  0.25659594 ... -0.23840512 -1.089901
   0.7021539 ]
 [ 0.38384575  0.3199892  -0.08682799 ...  0.5943302  -0.0815199
   0.96424836]
 [ 0.8728238   1.2109184   0.30578512 ...  0.2631717   1.7938929
   2.07715   ]] 
I 1: [[-1.6703042   0.04279047  1.1082757  ...  1.1201062  -1.0580032
  -0.18329015]
 [-1.3252518  -0.5656686   0.50778794 ...  1.8512449  -0.5306258
  -0.36595204]
 [-1.1735461  -0.89015764  0.26884112 ...  1.4333687  -0.44683903
  -0.42583343]
 ...
 [-0.9653343   0.10381357  0.25659594 ... -0.23840512 -1.089901
   0.7021539 ]
 [ 0.38384575  0.3199892  -0.08682799 ...  0.5943302  -0.0815199
   0.96424836]
 [ 0.8728238   1.2109184   0.30578512 

<Figure size 1200x1200 with 5 Axes>